## LHC papers
Resources:
* https://github.com/kamyu104/LeetCode-Solutions/blob/master/Python/h-index.py
* https://www.analyticsvidhya.com/blog/2017/09/building-machine-learning-model-fun-using-orange/
* https://orange.biolab.si/blog/2016/07/18/network-analysis-with-orange/

In [5]:
import pandas as pd
import time
import random

## Functions

In [6]:
def normalize(df,nd={'title':'title_TeX',
                     'doi':'DOI'}):
    for k in nd.keys():
        if k in df:
            df[nd[k]]=df[k].apply(
                    lambda d: d[0] if isinstance(d,list) else d
                )
            #special cases
            if k=='title':
                df[nd[k]]=df[nd[k]].apply(
                    lambda d: d.get(k) if isinstance(d,dict) else d)
    return df

In [7]:
lhc_query='cn cms or cn atlas or cn lhcb or cn alice'

## Get query results

In [8]:
q='{} and collection:published'.format(lhc_query)
q=q.replace(' ','+')
#df=pd.read_json(
#    'https://inspirehep.net/search?p=find+{}&rg=250&of=recjson'.format(q))

In [9]:
from urllib.error import HTTPError
def to_json_append(df,file):
    '''
    Load the file with
    pd.read_json(file,orient='records',lines=True)
    '''
    df.to_json('tmp.json',orient='records',lines=True)
    #append
    f=open('tmp.json','r')
    k=f.read()
    f.close()
    f=open(file,'a')
    f.write('\n') #Prepare next data entry
    f.write(k)
    f.close()
    
def get_query(q):
    cc=pd.DataFrame()
    jrec=cc.shape[0]
    while jrec%250==0:
        query='http://inspirehep.net/search?ln=en&p={}&rg=250&jrec={}&of=recjson'.format(
              q,jrec+1)
        try:
            c=pd.read_json(query)
        except:# HTTPError:
            print('WARNING → May be HTTPError: HTTP Error 404: Not Found')
            c=pd.DataFrame()
        c=normalize(c)
        if not c.empty:
            cc=(cc.append(c,sort=True)).reset_index(drop=True)
            jrec=cc.shape[0]
        else:
            jrec=jrec+1 # To exit the loop
        print('jrec',jrec)
        time.sleep(1)
    return cc

def get_citations(recid,sleep=3):
    cc=pd.DataFrame()
    jrec=cc.shape[0]
    while jrec%250==0:
        qcites='http://inspirehep.net/search?ln=en&p=refersto:recid:{}&rg=250&jrec={}&of=recjson'.format(
              recid,jrec+1)
        try:
            c=pd.read_json(qcites)
        except:# HTTPError:
            print('WARNING → May be HTTPError: HTTP Error 404: Not Found')
            c=pd.DataFrame()
        c=normalize(c)
        for k in ['recid','authors','DOI','publication_info']:
            if k not in c:
                c[k]=None

        if not c.empty:
            c['cites']=c.apply(lambda row: 
                               {'recid':row['recid'],
                                'authors':row['authors'],
                                'DOI'    :row['DOI'], #Year info:
                                'publication_info':row['publication_info'],
                                'imprint': row['imprint'],
                                'primary_report_number':row['primary_report_number'],
                                'creation_date':row['creation_date']
                               },   axis=1)
        if not c.empty:
            cc=(cc.append(c,sort=True)).reset_index(drop=True)
            #Temporal storage
            if 'cites' in cc:
                try:
                    tmpdf=cc.drop('cites',axis='columns')
                    #Be sure to start with empty 'tmp_cites.json'
                    to_json_append(tmpdf,'tmp_cites.json')
                except:
                    pass

            jrec=cc.shape[0]
        else:
            jrec=jrec+1 # To exit the loop
        time.sleep(sleep)
    return cc
def get_citations_list(recid):
    c=get_citations(recid)
    print(recid,c.shape[0])
    time.sleep(random.randint(1,5))
    if not c.empty:
        c[['cites']]
        return list(c['cites'].values)
    else:
        return None

In [10]:
RELOAD=False
if RELOAD:
    lhc=get_query(q)

In [7]:
if RELOAD:
    lhc.to_json('lhc.json.gz',compression='gzip')

In [11]:
lhc=pd.read_json('lhc.json.gz',compression='gzip')

In [12]:
lhc.shape

(3584, 45)

In [13]:
df=lhc.copy()

## Add citations metadata

Add citations column

In [14]:
df=df.sort_values('number_of_citations').reset_index(drop=True)

In [15]:
dff=df[df['number_of_citations']==0].reset_index(drop=True)
dfc=df[df['number_of_citations'] >0].reset_index(drop=True)

In [16]:
dfc.shape

(3292, 45)

In [17]:
nstr=500;nend=501
qq=dfc[nstr:nend].reset_index(drop=True)

In [18]:
s=time.time()
#df['cites']=df['recid'].apply(get_citations_list)
qq['cites']=qq['recid'].apply(get_citations_list)
tot=time.time()-s
print(tot)

637781 5
18.548848628997803


In [42]:
kk=qq.copy()
kk['nc']=kk['cites'].apply(lambda l: len(l) if isinstance(l,list) else 0)
dfy=kk[kk.apply(lambda row: row['number_of_citations']<=row['nc'],axis=1)].reset_index(drop=True)
dfn=kk[kk.apply(lambda row: row['number_of_citations']>row['nc'],axis=1)].reset_index(drop=True)
dfn.shape[0],dfn.shape[0]+dfy.shape[0],df.shape[0]

(2, 6, 3584)

In [44]:
FIX=False
if FIX:
    dff=(dff.append(qq,sort=True)).reset_index(drop=True)

In [39]:
sh=dff.shape[0]
if dfn.shape[0]==0:
    dff=(dff.append(qq,sort=True)).reset_index(drop=True)
    shend=dff.shape[0]
    print(sh,shend)
elif dfn.shape[0]>0:
    dff=(dff.append(dfy,sort=True)).reset_index(drop=True)
    qq=dfn.copy()
    print('some failed:',qq.shape[0])

some failed: 6


In [45]:
qq[qq.apply(lambda row: row['number_of_citations']>row['nc'],axis=1)][['nc','number_of_citations']]

,nc,number_of_citations
0,0,199
1,0,201
2,313,314
3,706,707
4,500,949
5,0,975


In [46]:
dff.to_json('dff_preliminar.json.gz',compression='gzip')

In [ ]:
del dff

In [13]:
#dff=pd.read_json('dff_preliminar.json.gz',compression='gzip')

In [15]:
dff.shape

(2799, 46)

In [1]:
journal={}

In [2]:
import time
def get_journal_info(row,journal=journal,abrv_journal='publication_info'
                                        ,abrv_journal_title='title'
                                        ,doi='doi'
                                        ,container_title='container_title'
                                        ,issn='ISSN'):
    
    try:
        j=row.get(abrv_journal).get(abrv_journal_title)
    except:
        j=None
    if j and not journal.get(j):
        journal.update( {j:{}} )
    
    if not journal.get(j) and row.get(doi):
        strdoi=row.get(doi)
        try:
            di=pd.read_json( 'https://api.crossref.org/works/{}'.format( strdoi  ) )
        except:
            di=pd.DataFrame()
        if j in journal and not journal.get(j).get(container_title):
            try:
                ct=di.loc['container-title'][0][0]
                journal[j].update({container_title:ct})

            except:
                pass
        if j in journal and not journal.get(j).get(issn):
            try:
                gis=di.loc['ISSN'].get('message')[0]
                journal[j].update({issn:gis})
            except:
                pass
        #Get a new journal crossref metadata here

        time.sleep(1)
        return di

In [75]:
di=dfp.apply(get_journal_info,axis=1)

In [76]:
journal

{'Eur.Phys.J.': {'ISSN': '1434-6044',
  'container_title': 'The European Physical Journal C'},
 'Europhys.Lett.': {'ISSN': '0295-5075',
  'container_title': 'Europhysics Letters (EPL)'},
 'Int.J.Mod.Phys.': {'ISSN': '0217-751X',
  'container_title': 'International Journal of Modern Physics A'},
 'J.Phys.': {'ISSN': '0954-3899',
  'container_title': 'Journal of Physics G: Nuclear and Particle Physics'},
 'JCAP': {'ISSN': '1475-7516',
  'container_title': 'Journal of Cosmology and Astroparticle Physics'},
 'JHEP': {'ISSN': '1029-8479',
  'container_title': 'Journal of High Energy Physics'},
 'JINST': {'ISSN': '1748-0221',
  'container_title': 'Journal of Instrumentation'},
 'Mod.Phys.Lett.': {'ISSN': '0217-7323',
  'container_title': 'Modern Physics Letters A'},
 'Nucl.Phys.': {'ISSN': '0550-3213', 'container_title': 'Nuclear Physics B'},
 'Phys.Lett.': {'ISSN': '0370-2693', 'container_title': 'Physics Letters B'},
 'Phys.Rept.': {'ISSN': '0370-1573', 'container_title': 'Physics Reports'

Normalize data

In [81]:
%%bash
git commit -am 'lc'
git push origin master

[master ba8b47f] lc
 1 file changed, 348 insertions(+), 6 deletions(-)


To github.com:restrepo/inspire.git
   0448bba..ba8b47f  master -> master
